In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)

## Cases and deaths
- by day and county
- prepared by Aleksanda

### CSV to Parquet conversion

In [ ]:
# pd.read_csv(
#     "../data/cases_daily.csv", 
#     usecols=["date", "fips", "cases", "deaths"]
# ).to_parquet(
#     "../data/cases_dailt.parquet",
#     engine="pyarrow", 
#     compression="brotli"
# )

In [9]:
df_cases = pd.read_parquet("../data/cases_dailt.parquet")
df_cases.head()

,date,fips,cases,deaths
0,2020-01-21,1001,0,0
1,2020-01-22,1001,0,0
2,2020-01-23,1001,0,0
3,2020-01-24,1001,0,0
4,2020-01-25,1001,0,0


In [14]:
client["MDAProjectDatabase"]["daily_covid_cases"].insert_many(df_cases.to_dict("records"))

## Socio-economic data

In [17]:
df_socio_economic = pd.read_csv("../data/non_epidemic_data.csv")
print(df_socio_economic.shape)
df_socio_economic.head()

(3233, 11)


,fips,area,pop_estimate_2019,unemployment_rate_2020,median_household_income_2019,poverty_frac_2019,no_high_school,high_school_only,college_only,bachelor_or_higher,voted_biden
0,1001,Autauga County,55869.0,4.9,58233.0,0.121,0.114834,0.335885,0.283566,0.265716,0.270184
1,1003,Baldwin County,223234.0,5.6,59871.0,0.101,0.091938,0.276596,0.312841,0.318625,0.224090
2,1005,Barbour County,24686.0,7.0,35972.0,0.271,0.267869,0.356045,0.260298,0.115787,0.457882
3,1007,Bibb County,22394.0,6.6,47918.0,0.203,0.209426,0.448788,0.238001,0.103785,0.206983
4,1009,Blount County,57826.0,4.1,52902.0,0.163,0.195094,0.334221,0.339750,0.130934,0.095694


In [18]:
fipses_cases = pd.read_csv("../data/county_daily_csv.csv", usecols=["fips"]).fips.values

df_socio_economic = df_socio_economic.loc[df_socio_economic.fips.isin(fipses_cases), :]
df_socio_economic.shape

(3217, 11)

In [20]:
client["MDAProjectDatabase"]["socio_economic_data"].insert_many(df_socio_economic.drop(columns=["area"]).to_dict("records"))

## FIPS dictionary

In [35]:
df_fips = pd.read_csv("../data/fips.csv")
df_fips.head()

,fips,area
0,1000,Alabama
1,1001,Autauga County
2,1003,Baldwin County
3,1005,Barbour County
4,1007,Bibb County


In [36]:
client["MDAProjectDatabase"]["fips_codes"].insert_many(df_fips.to_dict("records"))